In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os.path import join
from glob import glob
from collections import Counter
import torchvision

from utils.io import load_json
from paths import VIDEO_DIR

In [3]:
mode = "val"

json_path = f"./data/{mode}_data.json"
mode_data = load_json(json_path)

In [4]:
len(mode_data)

4180

In [5]:
mode_data[10]

{'num_segments': 6,
 'description': "a deep blue lighting display is projected upon the building's face.",
 'dl_link': 'https://www.flickr.com/video_download.gne?id=8328905760',
 'times': [[0, 0], [0, 0], [0, 0], [0, 0]],
 'video': '23748404@N00_8328905760_5e0200aae1.mov',
 'annotation_id': 32787}

In [6]:
# serach video

def search_video(video_name, video_dir):
    """Search for a video in DiDeMo dataset."""
    video_path = join(video_dir, video_name + "*")
    options = glob(video_path)
    assert len(options) > 0, f"No video found at {video_path}"
    assert len(options) < 2, f"Multiple videos found with name {video_name}"
    return options[0]

In [7]:
video_name = mode_data[0]["video"]
video_dir = VIDEO_DIR

video_path = search_video(video_name, video_dir)

In [8]:
# get majority vote annotation

def get_majority_vote(times: list):
    """
    Given list of times [(t1_s, t1_e), (t2_s, t2_e), .., (tK_s, tK_e)],
    find the tuple (t_s, t_e) s.t. it is a majority among the tuples.
    """
    times = [str(x) for x in times]
    c = Counter(times)
    most_common = c.most_common()
    return eval(most_common[0][0])

In [9]:
mode_data[0]["times"]

[[1, 1], [1, 1], [1, 1], [0, 0]]

In [10]:
get_majority_vote([[1,1], [2,2],[1,3],[1,3]])

[1, 3]

In [11]:
def process_time_annotation(annotation, video_duration, clip_duration=5.):
    """Returns actual timestamps from annotations."""
    t_start, t_end = annotation
    t_start = t_start * clip_duration
    t_end = max((t_end + 1) * clip_duration, video_duration)
    
    return t_start, t_end

In [12]:
video_stream = torchvision.io.VideoReader(video_path, "video")


# video_duration = 
# t_start, t_end = 

RuntimeError: Not compiled with video_reader support, to enable video_reader support, please install ffmpeg (version 4.2 is currently supported) and build torchvision from source.